### This is the test for data for learning from Thunder BOT

- P1 is human, we are learning P2 (P2 is Thunder BOT).

- Some frames will be removed, there are frames that;
    - Both players standing.
    - If we are(P2) in RECOV frame(Since we don't make RECOV frames ourselves).

### Importing libraries

In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import TensorBoard
from time import time
import numpy as np

# For using core package that located in the two upper folder.
import sys
sys.path.append('../../')

from core.json_importer import parse_json_file
from core.filters import remove_unpressed_frames, remove_both_standing_frames, remove_same_consecutive_actions, remove_recov_frames
from core.actions import one_hot_encode, decode
from core.preproccessing import Normalizer
from core.helpers import write_file

Using TensorFlow backend.


In [2]:
# Open file
game_data = parse_json_file("data/VSTHUNDER.json")

### Pre process data

- In the pre process phase, we remove P2's datas from training set in order to crate labels. Labels are just actions of P2.

- We calculate the distance for each coordinate axis. (x distance, y distance)

- We also make actions one-hot encoding. 

- One-hot encoding for inputs and labels are not same. For example, Dash action in the input may have encoding [0,1] while Dash action in the label have encoding [0, 0, 1, 0].

- Integer values normalized. (P1-HP, P2-HP, X-Distance, Y-Distance)

- <font color='green'>There must be mechanism for understanding is player closing or move away.</font>

- There is no test and validation set preparation. Data that gathered from different rounds can be used for this problem.

- After pre-process we just have <font color='red'>[P1-Action, P1-HP, P2-HP, X-Distance, Y-Distance] -> P2-Action</font>

#### Remove unneeded frames

In [3]:
l1 = game_data.filter(remove_both_standing_frames)
l3 = game_data.filter(remove_recov_frames)

print("Removed {} frames that both players are standing".format(l1))
print("Removed {} frames that are recov frames".format(l3))
print("\n{} frames removed.".format(l1 + l3))
print("{} frames left.".format(len(game_data)))

Removed 258 frames that both players are standing
Removed 2419 frames that are recov frames

2677 frames removed.
8088 frames left.


#### Encoding and normalization

In [4]:
def normalize_and_save(data, file_name):
    p1_hp_normalizer = Normalizer()
    p1_normalized_hp = p1_hp_normalizer.normalize(data)
    p1_hp_normalizer.save("out/" + file_name)
    return p1_normalized_hp

## Pre process data
processed_data = []

# Create one hot encoding for actions (For input and labels)
p1_one_hot_encoded_actions = one_hot_encode(game_data.get_column("P1-action"))
labels = one_hot_encode(game_data.get_column("P2-action"))

# Normalize uncategorized features
p1_normalized_hp = normalize_and_save(game_data.get_column("P1-hp"), "p1_hp_norm.save")
p2_normalized_hp = normalize_and_save(game_data.get_column("P2-hp"), "p2_hp_norm.save")
normalized_x_distance = normalize_and_save([frame["P1-x"] - frame["P2-x"] for frame in game_data], "x_norm.save")
normalized_y_distance = normalize_and_save([frame["P1-y"] - frame["P2-y"] for frame in game_data], "y_norm.save")

for index in range(len(game_data)):    
    processed_row = []
    processed_row.extend(p1_one_hot_encoded_actions[index])
    processed_row.extend(p1_normalized_hp[index]) 
    processed_row.extend(p2_normalized_hp[index])
    processed_row.extend(normalized_x_distance[index])
    processed_row.extend(normalized_y_distance[index])
    
    processed_data.append(processed_row)
processed_data = np.array(processed_data)
labels = np.array(labels)

### Example data

In [5]:
EXAMPLE_ROW = 10

print("There are %d frames in dataset." % len(processed_data))
print("After pre processing the shape of our dataset is %s" % str(processed_data.shape))
print("\nOne example in index %d." % EXAMPLE_ROW)

row = processed_data[EXAMPLE_ROW]
print("\tProcessed Frame:" )
print("\t\tP1 Action(one-hot):\t%s" % row[0: 55])
print("\t\tP1 Hp(norm.):\t\t%s" % row[55])
print("\t\tP2 Hp(norm.):\t\t%s" % row[56])
print("\t\tX Dist(norm.):\t\t%s" % row[57])
print("\t\tY Dist(norm.):\t\t%s" % row[58])
print("\t\tLabel:\t\t\t%s" % labels[EXAMPLE_ROW])

There are 8088 frames in dataset.
After pre processing the shape of our dataset is (8088, 59)

One example in index 10.
	Processed Frame:
		P1 Action(one-hot):	[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
		P1 Hp(norm.):		0.0
		P2 Hp(norm.):		0.0
		X Dist(norm.):		0.10673443456162643
		Y Dist(norm.):		0.035483870967741936
		Label:			[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


### Neural Network Design

Our neural network has two hidden layers in this test. They has 12 and 8 neurons respectively.

In [6]:
# Constants
INPUT_LAYER_SIZE = processed_data.shape[1]
OUTPUT_LAYER_SIZE = labels.shape[1]

In [7]:
model = Sequential()
model.add(Dense(12, input_dim=INPUT_LAYER_SIZE, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(OUTPUT_LAYER_SIZE, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
# Logging for tensorboard
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

In [9]:
model.fit(processed_data, labels, epochs=150, batch_size=10, callbacks=[tensorboard])

Epoch 1/150
8088/8088 [==============================] - 1s 139us/step - loss: 0.1339 - acc: 0.9618
Epoch 2/150
8088/8088 [==============================] - 1s 115us/step - loss: 0.0479 - acc: 0.9825
Epoch 3/150
8088/8088 [==============================] - 1s 114us/step - loss: 0.0413 - acc: 0.9862
Epoch 4/150
8088/8088 [==============================] - 1s 103us/step - loss: 0.0380 - acc: 0.9872
Epoch 5/150
8088/8088 [==============================] - 1s 104us/step - loss: 0.0366 - acc: 0.9875
Epoch 6/150
8088/8088 [==============================] - 2s 228us/step - loss: 0.0356 - acc: 0.9876
Epoch 7/150
8088/8088 [==============================] - 1s 140us/step - loss: 0.0348 - acc: 0.9880
Epoch 8/150
8088/8088 [==============================] - 1s 121us/step - loss: 0.0340 - acc: 0.9882
Epoch 9/150
8088/8088 [==============================] - 1s 105us/step - loss: 0.0333 - acc: 0.9882
Epoch 10/150
8088/8088 [==============================] - 1s 104us/step - loss: 0.0325 - acc: 0.9884

In [10]:
scores = model.evaluate(processed_data, labels)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

8088/8088 [==============================] - 0s 17us/step

acc: 99.16%


### Save model

In [11]:
model.save("out/model.h5")
write_file("out/config.json", model.to_json())

In [12]:
game_data.rounds[0][0][game_data.columns.index("P2-action")]

'FORWARD_WALK'